In [1]:
import numpy as np
import pandas as pd
import random
import time
random.seed(time.time())

In [15]:
class RNNConfig():
    __data__ = dict(
        input_size=5,
        num_steps=30,
        lstm_size=32,

        num_layers=1,
        keep_prob=0.8,
        batch_size = 200,
        init_learning_rate = 0.05,
        learning_rate_decay = 0.99,
        init_epoch = 5,
        max_epoch = 500,

        embedding_size = 8,
        sector_size = None,
        stock_symbol_size = None,
    )

    def to_dict(self):
        return self.__data__
    
    def __str__(self):
        return str(self.to_dict())

    def __repr__(self):
        return str(self.to_dict())
    
    def __setattr__(self, name, val):
        self.__data__[name] = val
    
    def __getattr__(self, name):
        return self.__data__[name]

config = RNNConfig()
config.sector_size = -1
config.to_dict()

{'batch_size': 200,
 'embedding_size': 8,
 'init_epoch': 5,
 'init_learning_rate': 0.05,
 'input_size': 5,
 'keep_prob': 0.8,
 'learning_rate_decay': 0.99,
 'lstm_size': 32,
 'max_epoch': 500,
 'num_layers': 1,
 'num_steps': 30,
 'sector_size': -1,
 'stock_symbol_size': None}

In [16]:
learning_rates_to_use = [
    config.init_learning_rate * (
        config.learning_rate_decay ** max(float(i + 1 - config.init_epoch), 0.0)
    ) for i in range(config.max_epoch)]
print learning_rates_to_use[:20]
print 'middle:', learning_rates_to_use[len(learning_rates_to_use) // 2]
print learning_rates_to_use[-20:]

[0.05, 0.05, 0.05, 0.05, 0.05, 0.0495, 0.049005, 0.04851495, 0.0480298005, 0.047549502495, 0.04707400747005, 0.0466032673953495, 0.046137234721396005, 0.045675862374182044, 0.04521910375044022, 0.04476691271293582, 0.04431924358580647, 0.04387605114994839, 0.043437290638448915, 0.043002917732064425]
middle: 0.00421917826632
[0.00041814088372523514, 0.0004139594748879828, 0.00040981988013910294, 0.0004057216813377119, 0.00040166446452433483, 0.0003976478198790915, 0.00039367134168030054, 0.0003897346282634975, 0.00038583728198086253, 0.0003819789091610539, 0.00037815912006944336, 0.00037437752886874894, 0.0003706337535800614, 0.00036692741604426086, 0.0003632581418838182, 0.00035962556046498, 0.00035602930486033023, 0.00035246901181172695, 0.00034894432169360964, 0.00034545487847667354]


In [17]:
class StockDataSet(object):
    def __init__(self, 
                 stock_sym, 
                 stock_sector,
                 config,
                 test_ratio=0.1,
                 normalized=True,
                 close_price_only=True,
                 **kwargs):
        
        self.stock_sym = stock_sym
        self.stock_sector = stock_sector

        self.input_size = config.input_size
        self.num_steps = config.num_steps
        self.test_ratio = test_ratio
        self.close_price_only = close_price_only
        self.normalized = normalized
        
        # Read csv file
        raw_df = pd.read_csv("_data/%s.csv" % stock_sym)
        
        # Merge into one sequence
        if close_price_only:
            self.raw_seq = raw_df['Close'].tolist()
        else:
            self.raw_seq = [price for tup in raw_df[['Open', 'Close']].values for price in tup]
        
        self.train_X, self.train_y, self.test_X, self.test_y = self._prepare_data(self.raw_seq)

    def _prepare_data(self, seq):
        # split into items of input_size
        seq = [np.array(seq[i * self.input_size: (i + 1) * self.input_size]) 
               for i in range(len(seq) // self.input_size)]

        if self.normalized:
            seq = [seq[0] / seq[0][0] - 1.0] + [
                curr / seq[i - 1][-1] - 1.0 for i, curr in enumerate(seq[1:])]
        
        # split into groups of num_steps
        X = np.array([seq[i: i + self.num_steps] for i in range(len(seq) - self.num_steps)])
        y = np.array([seq[i + self.num_steps] for i in range(len(seq) - self.num_steps)])

        train_size = int(len(X) * (1.0 - self.test_ratio))
        train_X, test_X = X[:train_size], X[train_size:]
        train_y, test_y = y[:train_size], y[train_size:]
        return train_X, train_y, test_X, test_y
    
    def generate_one_epoch(self, batch_size):
        num_batches = int(len(self.train_X)) // batch_size
        if batch_size * num_batches < len(self.train_X):
            num_batches += 1
        
        batch_indices = range(num_batches)
        random.shuffle(batch_indices)
        for j in batch_indices:
            batch_X = self.train_X[j * batch_size: (j+1) * batch_size]
            batch_y = self.train_y[j * batch_size: (j+1) * batch_size]
            assert set(map(len, batch_X)) == {self.num_steps}
            yield batch_X, batch_y

In [18]:
import os
SP500_symbols = pd.read_csv("_data/constituents-financials.csv")
SP500_symbols.rename(columns={col: col.lower() for col in SP500_symbols.columns}, inplace=True)
SP500_symbols['file_exists'] = SP500_symbols['symbol'].map(lambda x: os.path.exists("_data/{}.csv".format(x)))

print SP500_symbols['file_exists'].value_counts()
SP500_symbols = SP500_symbols[SP500_symbols['file_exists']]

print SP500_symbols.shape
SP500_symbols.head()

False    381
True     124
dtype: int64
(124, 16)


,symbol,name,sector,price,dividend yield,price/earnings,earnings/share,book value,52 week low,52 week high,market cap,ebitda,price/sales,price/book,sec filings,file_exists
4,ATVI,Activision Blizzard,Information Technology,48.06,0.64,37.55,1.28,12.23,30.37,48.36,36.13,2.14000,5.44,3.91,http://www.sec.gov/cgi-bin/browse-edgar?action...,True
6,ADBE,Adobe Systems Inc,Information Technology,119.98,0.00,51.72,2.32,15.02,83.25,120.69,59.28,1.82000,10.14,8.00,http://www.sec.gov/cgi-bin/browse-edgar?action...,True
14,AKAM,Akamai Technologies Inc,Information Technology,63.30,NaN,35.36,1.79,18.61,47.80,71.64,10.96,0.70798,4.67,3.39,http://www.sec.gov/cgi-bin/browse-edgar?action...,True
17,ALXN,Alexion Pharmaceuticals,Health Care,129.18,NaN,73.40,1.76,38.81,109.12,162.00,29.02,1.17000,9.71,3.44,http://www.sec.gov/cgi-bin/browse-edgar?action...,True
23,GOOGL,Alphabet Inc Class A,Information Technology,851.15,NaN,30.53,27.88,201.12,672.66,867.00,588.50,29.86000,6.49,4.21,http://www.sec.gov/cgi-bin/browse-edgar?action...,True


In [19]:
syms = SP500_symbols['symbol'].tolist()
sym_to_sector = {row['symbol']: row['sector'] for _, row in SP500_symbols.iterrows()}
stock_data_by_sym = {sym: StockDataSet(sym, sym_to_sector[sym], config, test_ratio=0.1) for sym in syms}

In [20]:
print stock_data_by_sym['GOOGL'].stock_sector
stock_data_by_sym['GOOGL'].test_y[:5]

Information Technology


array([[-0.04555013, -0.08137981, -0.09448573, -0.08838159, -0.11025901],
       [-0.08100061, -0.10406455, -0.2008013 , -0.186304  , -0.20059044],
       [-0.12080947, -0.14634922, -0.14260186, -0.13242629, -0.11763861],
       [ 0.02430099,  0.03201664,  0.03122521,  0.02449882,  0.02545501],
       [ 0.04109767,  0.03384512,  0.01149953, -0.00032668,  0.02561252]])

In [21]:
uniq_sectors = SP500_symbols['sector'].value_counts().index.tolist()
sector_to_index = {s: idx for idx, s in enumerate(uniq_sectors)}

uniq_stock_symbols = SP500_symbols['symbol'].value_counts().index.tolist()
symbol_to_index = {s: idx for idx, s in enumerate(uniq_stock_symbols)}

config.sector_size = len(sector_to_index)
config.stock_symbol_size = len(uniq_stock_symbols)
print config.to_dict()

SP500_symbols['stock_symbol_index'] = SP500_symbols['symbol'].map(lambda x: symbol_to_index[x])
SP500_symbols['sector_index'] = SP500_symbols['sector'].map(lambda x: sector_to_index[x])
SP500_symbols[['name', 'symbol', 'stock_symbol_index', 'sector', 'sector_index']].head()

{'sector_size': 8, 'embedding_size': 8, 'max_epoch': 500, 'keep_prob': 0.8, 'lstm_size': 32, 'num_steps': 30, 'stock_symbol_size': 124, 'batch_size': 200, 'init_learning_rate': 0.05, 'num_layers': 1, 'input_size': 5, 'init_epoch': 5, 'learning_rate_decay': 0.99}


,name,symbol,stock_symbol_index,sector,sector_index
4,Activision Blizzard,ATVI,114,Information Technology,0
6,Adobe Systems Inc,ADBE,28,Information Technology,0
14,Akamai Technologies Inc,AKAM,93,Information Technology,0
17,Alexion Pharmaceuticals,ALXN,76,Health Care,2
23,Alphabet Inc Class A,GOOGL,36,Information Technology,0


---

In [22]:
import numpy as np
import random
import tensorflow as tf

tf.reset_default_graph()
lstm_graph = tf.Graph()

with lstm_graph.as_default():
    """
    The model asks for three things:
    - input: training data X
    - targets: training label y
    - learning_rate: 
    """
    learning_rate = tf.placeholder(tf.float32, None, name="learning_rate")
    stock_labels = tf.placeholder(tf.int32, [None,], name='stock_labels')  # mapped to an integer.

    # Number of examples, number of input, dimension of each input
    inputs = tf.placeholder(tf.float32, [None, config.num_steps, config.input_size], name="inputs")
    targets = tf.placeholder(tf.float32, [None, config.input_size], name="targets")
    
    def _create_one_cell():
        lstm_cell = tf.contrib.rnn.LSTMCell(config.lstm_size, state_is_tuple=True)
        if config.keep_prob < 1.0:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=config.keep_prob)
        return lstm_cell

    cell = tf.contrib.rnn.MultiRNNCell(
        [_create_one_cell() for _ in range(config.num_layers)], 
        state_is_tuple=True
    ) if config.num_layers > 1 else _create_one_cell()

    val, state_ = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, scope="lilian_rnn")

    print "Before transpose - val.get_shape() =", val.get_shape()

    # Before transpose, val.get_shape() = (batch_size, num_steps, lstm_size)
    # After transpose, val.get_shape() = (num_steps, batch_size, lstm_size)
    val = tf.transpose(val, [1, 0, 2])
    print "After transpose - val.get_shape() =", val.get_shape()

    embedding_matrix = tf.Variable(
        tf.random_uniform([config.stock_symbol_size, config.embedding_size], -1.0, 1.0),
        name="embedding_matrix"
    )
    # industries_embed.shape = (batch_size, embedding_size)
    stock_label_embeds = tf.nn.embedding_lookup(embedding_matrix, stock_labels)
    print stock_label_embeds.get_shape()

    with tf.name_scope("output_layer"):
        # last_state.get_shape() = (batch_size, lstm_size)
        last_state = tf.gather(val, int(val.get_shape()[0]) - 1, name="last_LSTM_state")
        print "last_state.get_shape() =", last_state.get_shape()
        
        # After concat, last_state_with_embed.get_shape() = (batch_size, lstm_size + embedding_size)
        last_state_with_embed = tf.concat([last_state, stock_label_embeds], axis=1, 
                                          name="last_LSTM_state_with_embed")

        weight = tf.Variable(
            tf.truncated_normal([config.lstm_size + config.embedding_size, config.input_size]), 
            name="lilian_weights"
        )
        bias = tf.Variable(tf.constant(0.1, shape=[config.input_size]), name="lilian_biases")
        print "last_state_with_embed.get_shape() =", last_state_with_embed.get_shape()
        print "weight.get_shape() =", weight.get_shape()
        print "bias.get_shape() =", bias.get_shape()
        prediction = tf.matmul(last_state_with_embed, weight) + bias

        tf.summary.histogram("last_LSTM_state", last_state)
        tf.summary.histogram("last_LSTM_state_with_embed", last_state_with_embed)
        tf.summary.histogram("weights", weight)
        tf.summary.histogram("biases", bias)

    with tf.name_scope("train"):
        # loss = -tf.reduce_sum(targets * tf.log(tf.clip_by_value(prediction, 1e-10, 1.0)))
        loss = tf.reduce_mean(tf.square(prediction - targets), name="loss_mse")
        optimizer = tf.train.AdamOptimizer(learning_rate)
        minimize = optimizer.minimize(loss, name="loss_mse_adam_minimize")
        tf.summary.scalar("loss_mse", loss)
    
    # operators you want to use after restoring the model
    for op in [prediction, loss]:
        tf.add_to_collection('ops_to_restore', op)

    #mistakes = tf.not_equal(tf.argmax(targets, 1), tf.argmax(prediction, 1))
    #error = tf.reduce_mean(tf.cast(mistakes, tf.float32))


Before transpose - val.get_shape() = (?, 30, 32)
After transpose - val.get_shape() = (30, ?, 32)
(?, 8)
last_state.get_shape() = (?, 32)
last_state_with_embed.get_shape() = (?, 40)
weight.get_shape() = (40, 5)
bias.get_shape() = (5,)


In [23]:
# Merged test data
merged_test_X = []
merged_test_y = []
merged_stock_sector_indices = []
merged_stock_symbol_indices = []

for symbol, stock_dataset in stock_data_by_sym.iteritems():
    stock_sector_index = sector_to_index[stock_dataset.stock_sector]
    stock_symbol_index = symbol_to_index[stock_dataset.stock_sym]
    
    test_size = len(stock_dataset.test_X)
    
    merged_test_X += list(stock_dataset.test_X)
    merged_test_y += list(stock_dataset.test_y)
    merged_stock_sector_indices += [stock_sector_index] * test_size
    merged_stock_symbol_indices += [stock_symbol_index] * test_size


In [24]:
merged_test_X = np.array(merged_test_X)
merged_test_y = np.array(merged_test_y)
merged_stock_sector_indices = np.array(merged_stock_sector_indices)
merged_stock_symbol_indices = np.array(merged_stock_symbol_indices)

print merged_test_X.shape
print merged_test_y.shape
print merged_stock_sector_indices.shape
print merged_stock_symbol_indices.shape

(13877, 30, 5)
(13877, 5)
(13877,)
(13877,)


In [36]:
import os
def _mkdir_if_not_exist(dir_name):
    if not os.path.exists(dir_name):
        print "Create folder:", dir_name
        os.mkdir(dir_name)

In [ ]:
import os
import random
import json

from tensorflow.contrib.tensorboard.plugins import projector


final_prediction = []
final_loss = None

graph_name = "All_lr%.2f_lr_decay%.3f_lstm%d_step%d_input%d_batch%d_epoch%d_symbol_embed%d" % (
    config.init_learning_rate, config.learning_rate_decay, 
    config.lstm_size, config.num_steps, 
    config.input_size, config.batch_size, config.max_epoch,
    config.embedding_size)

print "Graph Name:", graph_name

log_dir = '_logs/' + graph_name
model_dir = "_models/" + graph_name

_mkdir_if_not_exist(log_dir)
_mkdir_if_not_exist(model_dir)

with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as fout:
    print >> fout, "symbol" + "\t" + "sector"
    for stock_sym in sorted(symbol_to_index, key=lambda x: symbol_to_index[x]):
        print >> fout, stock_sym + "\t" + stock_data_by_sym[stock_sym].stock_sector

with tf.Session(graph=lstm_graph) as sess:
    saver = tf.train.Saver()

    merged_summary = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
    summary_writer.add_graph(sess.graph)

    ######################################################
    ### Set up embedding visualization
    # Format: tensorflow/tensorboard/plugins/projector/projector_config.proto
    projector_config = projector.ProjectorConfig()

    # You can add multiple embeddings. Here we add only one.
    added_embedding = projector_config.embeddings.add()
    added_embedding.tensor_name = embedding_matrix.name
    # Link this tensor to its metadata file (e.g. labels).
    added_embedding.metadata_path = os.path.join(log_dir, 'metadata.tsv')
    
    # The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
    # read this file during startup.
    projector.visualize_embeddings(summary_writer, projector_config)
    ######################################################

    tf.global_variables_initializer().run()
    
    test_data_feed = {
        inputs: merged_test_X,
        targets: merged_test_y,
        learning_rate: 0.0,
        stock_labels: merged_stock_symbol_indices,
    }

    for epoch_step in range(config.max_epoch):
        current_lr = learning_rates_to_use[epoch_step]
        
        for symbol, stock_dataset in stock_data_by_sym.iteritems():
            stock_sector_index = sector_to_index[stock_dataset.stock_sector]
            stock_symbol_index = symbol_to_index[stock_dataset.stock_sym]
            
            for batch_X, batch_y in stock_dataset.generate_one_epoch(config.batch_size):
                train_data_feed = {
                    inputs: batch_X, 
                    targets: batch_y, 
                    learning_rate: current_lr,
                    stock_labels: np.array([stock_symbol_index] * len(batch_X))
                }
                train_loss, _ = sess.run([loss, minimize], train_data_feed)
        
        if epoch_step % 5 == 0:
            test_loss, _pred, _summary = sess.run([loss, prediction, merged_summary], test_data_feed)
            assert len(_pred) == len(merged_test_y)
            print "Epoch %d [%f]:" % (epoch_step, current_lr), test_loss
            if epoch_step % 50 == 0:
                print "Predictions:", [(
                    map(lambda x: round(x, 4), _pred[-j]), 
                    map(lambda x: round(x, 4), merged_test_y[-j])
                ) for j in range(5)]

            # Save a checkpoint
            saver.save(sess, os.path.join(log_dir, "model.ckpt"), epoch_step)
            summary_writer.add_summary(_summary, global_step=epoch_step)
    
    print "Final Results:"
    final_prediction, final_loss = sess.run([prediction, loss], test_data_feed)
    print final_prediction, final_loss

    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    saver.save(sess, "{0}/stock_rnn_model_{0}.ckpt".format(model_dir), global_step=epoch_step)


with open("final_predictions.{}.json".format(graph_name), 'w') as fout:
    fout.write(json.dumps(final_prediction.tolist()))


Graph Name: All_lr0.05_lr_decay0.990_lstm32_step30_input5_batch200_epoch500_symbol_embed8
Create folder: _logs/All_lr0.05_lr_decay0.990_lstm32_step30_input5_batch200_epoch500_symbol_embed8
Create folder: _models/All_lr0.05_lr_decay0.990_lstm32_step30_input5_batch200_epoch500_symbol_embed8
Epoch 0 [0.050000]: 0.00939097
Predictions: [([0.035, 0.0236, 0.0169, 0.0221, 0.0165], [0.0161, 0.0, -0.0161, -0.0161, 0.0]), ([0.0798, 0.0516, 0.0316, 0.0545, 0.0588], [0.0727, 0.0909, 0.1091, 0.1091, 0.1273]), ([-0.0346, -0.0477, -0.0085, -0.0215, -0.0155], [0.0182, 0.0364, 0.0364, 0.0364, 0.0727]), ([0.0331, 0.0209, 0.0175, 0.0348, 0.0043], [0.0182, 0.0, -0.0182, 0.0, 0.0]), ([0.0216, 0.0468, 0.0031, 0.028, 0.0089], [0.0189, 0.0377, 0.0, 0.0189, 0.0377])]
Epoch 5 [0.049500]: 0.331137
Epoch 10 [0.047074]: 0.429627
Epoch 15 [0.044767]: 0.199325
Epoch 20 [0.042573]: 0.220103
Epoch 25 [0.040486]: 0.141822
Epoch 30 [0.038502]: 0.0862431
Epoch 35 [0.036615]: 0.284868
Epoch 40 [0.034821]: 0.0305238
Epoch 

In [ ]:
# Load a trained model
import tensorflow as tf

test_prediction = None
test_loss = None

with tf.Session() as sess: 
    # First let's load meta graph and restore weights
    #saver.restore(sess, tf.train.latest_checkpoint('_models/stock_rnn_model_{}.ckpt-499.meta'.format(name)))
    saver = tf.train.import_meta_graph('_models/stock_rnn_model_{}.ckpt-499.meta'.format(name))
    saver.restore(sess, tf.train.latest_checkpoint('_models/.'))

    graph = tf.get_default_graph()
    print graph.get_operations
    print sess.graph

    test_feed_dict = {
        graph.get_tensor_by_name('inputs:0'): SP500_dataset.test_X, 
        graph.get_tensor_by_name('targets:0'): SP500_dataset.test_y, 
        graph.get_tensor_by_name('learning_rate:0'): 0.0
    }
    
    ops = graph.get_collection('ops_to_restore')
    print ops

    prediction = graph.get_tensor_by_name('output_layer/add:0')
    loss = graph.get_tensor_by_name('train/loss_mse:0')

    test_prediction, test_loss = sess.run([prediction, loss], test_feed_dict)
    print test_prediction, test_loss

In [38]:
print merged_test_y[:10]
print merged_stock_symbol_indices[:10]
print final_prediction[:10]

[[ 0.01612903  0.         -0.01612903 -0.01612903  0.        ]
 [ 0.          0.03125     0.03125     0.046875    0.03125   ]
 [ 0.06451613  0.01612903  0.          0.01612903 -0.01612903]
 [-0.1969697  -0.18181818 -0.18181818 -0.16666667 -0.16666667]
 [-0.09836066 -0.14754098 -0.1147541  -0.09836066 -0.1147541 ]
 [ 0.          0.01818182  0.          0.01818182  0.01818182]
 [ 0.03703704  0.01851852  0.07407407  0.09259259  0.09259259]
 [ 0.05357143  0.08928571  0.07142857  0.08928571  0.05357143]
 [-0.01694915 -0.03389831 -0.03389831  0.05084746  0.08474576]
 [ 0.08474576  0.08474576  0.08474576  0.05084746  0.05084746]]
[34 34 34 34 34 34 34 34 34 34]
[[-0.02814335 -0.02744995 -0.02656104 -0.02613362 -0.02431634]
 [ 0.00884791  0.00895659  0.00982992  0.00984628  0.01205399]
 [ 0.0466904   0.04632494  0.04728033  0.04821865  0.04977461]
 [-0.03051527 -0.02972901 -0.02864716 -0.02775948 -0.02567241]
 [-0.06001961 -0.05907817 -0.05899962 -0.05880501 -0.05733833]
 [-0.02295013 -0.02232

In [39]:
sym_to_final_prediction = {}
for sym_label, sym_index in symbol_to_index.iteritems():
    target_indices = np.array([
        _idx for _idx, _sym_index in enumerate(merged_stock_symbol_indices) 
        if _sym_index == sym_index])
    sym_to_final_prediction[sym_label] = final_prediction[target_indices]

In [ ]:
def _flatten(seq):
    return [x for y in seq for x in y]

def _denormalize(raw_seq, test_y):
    base_seq_index = len(SP500_dataset.train_X)
    print "start_point:", raw_seq[base_seq_index]
    denorm_test_y = []
    for seq in test_y:
        denorm_seq = (np.array(seq) + 1.0) * raw_seq[base_seq_index]
        base_seq_index += 1
        denorm_test_y += list(denorm_seq)
    return denorm_test_y

def plot_test(truths, preds, xlab=None, ylab=None, title=None):
    days = range(len(truths))
    print title, 'Total num. of days:', len(days)
    #plt.figure(figsize=(8, 6))
    plt.plot(days, truths, label='truth')
    plt.plot(days, preds, label='pred')
    plt.legend()
    plt.xlabel(xlab or "day")
    plt.ylabel(ylab or "normalized price")
    plt.grid(ls='--')
    if title:
        plt.title(title)
    plt.show()

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 18})

selected_symbols = random.sample(sym_to_final_prediction.keys(), 5)

for sym in selected_symbols:
    sym_final_prediction = sym_to_final_prediction[sym]
    sym_test_y = stock_data_by_sym[sym].test_y

    plot_test(_flatten(sym_test_y)[:200], _flatten(sym_final_prediction)[:200], 
              xlab='first 200 days in test data', title=sym)
    plot_test(_flatten(sym_test_y)[-200:], _flatten(sym_final_prediction)[-200:], 
              xlab='last 200 days in test data', title=sym)
    plot_test(_flatten(sym_test_y), _flatten(sym_final_prediction), title=sym)


In [ ]:
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt

plotting_date = pd.read_csv("_data/SP500.csv").tail(1000)
plotting_date['Date'] = pd.to_datetime(plotting_date['Date'])
print plotting_date.head()
plt.plot_date(plotting_date['Date'], plotting_date['Close'], '-', color='#e01f1f')
plt.ylabel('Daily S&P 500 Close Price')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.subplots_adjust(bottom=0.25)
plt.grid(ls=':', alpha=0.3)
plt.show()

In [ ]:
# One step: input_size=3, num_steps=2
original [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
input_size=3 [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
len(seq) = 4
apply num_steps=2
X = [
    [[1, 2, 3], [4, 5, 6]],
    [[4, 5, 6], [7, 8, 9]],
]
y = [
    [7, 8, 9],
    [10, 11, 12],
]

    # split into groups of num_steps
    X = np.array([seq[i: i + self.num_steps] for i in range(len(seq) - self.num_steps)])
    y = np.array([seq[i + self.num_steps] for i in range(len(seq) - self.num_steps)])
